# ch10. 데이터베이스 연동

## 1절. SQLite 데이터베이스연결

- SQLite 데이터베이스는 별도의 DBMS 없이 SQL을 이용하여 DB 액세스할 수 있도록 만든 **간단한 디스크기반의 데이터베이스를 제공**하는 C 라이브러리
- SQLite는 프로토타입(시제품이 나오기 전까지의 제품)을 만들때 사용. 정식 제품은 오라클이나 다른 DBMS를 이식
- [DB browser for SQLite](http://sqlitebrowser.org/dl)

### 1.1 SQLite browser 설치 및 sqlite3 패키지 load

In [132]:
import sqlite3
sqlite3.sqlite_version

'3.35.4'

In [133]:
import pandas as pd
pd.__version__

'1.2.4'

### 1.2 데이터베이스 연결

In [134]:
# DB 연결 생성시, 파일이 없으면 그 파일 연결. 파일이 없으면 빈 파일을 생성
conn=sqlite3.connect('data/ch10_example.db')
conn

In [135]:
#커서 객체 반환. 커서는 SQL문 실행시키고 결과데이터(SELECT)를 조회하는데 사용하는 객체
cursor = conn.cursor()
cursor

In [136]:
cursor.execute("""
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
""")

OperationalError: table MEMBER already exists

In [137]:
cursor.execute("""
    DROP TABLE MEMBER
""")

In [138]:
cursor.execute("""
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
""")

In [139]:
cursor.execute("INSERT INTO MEMBER VALUES ('홍길동', 20 , 'H@H.COM')")
cursor.execute("INSERT INTO MEMBER VALUES (\'홍길동\', 20 , \'H@H.COM\')")
cursor.execute("INSERT INTO MEMBER VALUES ('홍길동', 20 , 'H@H.COM')")
conn.commit() # 反 conn.rollback() #DML에서만 사용

In [140]:
cursor.arraysize
conn.rollback()

In [141]:
# select 실행결과는 cursor가 가리킴
cursor.execute('SELECT * FROM MEMBER')

In [142]:
cursor.arraysize

1

In [143]:
# select 문의 결과를 받는 함수
## fetchall() - 결과를 모두 받을 때(튜플 list)
## fetchone() - 결과를 한행씩 받을 때(튜플)
## fetchmany(n) - 결과를 n행 받을 때
print(cursor.fetchall()) #한번 소요된 cursor객체는 다시 fetch할 수 없음.

[('홍길동', 20, 'H@H.COM'), ('홍길동', 20, 'H@H.COM'), ('홍길동', 20, 'H@H.COM')]


In [144]:
#그래서 변수로 담아 놓음
cursor.execute('SELECT * FROM MEMBER')
members = cursor.fetchall()

In [145]:
for member in members:
    print(member)

('홍길동', 20, 'H@H.COM')
('홍길동', 20, 'H@H.COM')
('홍길동', 20, 'H@H.COM')


In [146]:
cursor.execute('SELECT * FROM MEMBER')
member_list=[]
while True:
    member = cursor.fetchone()
    if member is None:
        break
    member_list.append({'name':member[0],'age':member[1],'email':member[2]})
print(member_list)

[{'name': '홍길동', 'age': 20, 'email': 'H@H.COM'}, {'name': '홍길동', 'age': 20, 'email': 'H@H.COM'}, {'name': '홍길동', 'age': 20, 'email': 'H@H.COM'}]


In [147]:
cursor.execute('SELECT * FROM MEMBER ORDER BY AGE DESC')
for member in cursor.fetchmany(2):
    print(member)

('홍길동', 20, 'H@H.COM')
('홍길동', 20, 'H@H.COM')


In [148]:
print(cursor.fetchmany(size=cursor.arraysize))

[('홍길동', 20, 'H@H.COM')]


In [149]:
cursor.close() #cursor.close는 생략 가능
conn.close()

### 1.3 SQL 구문에 파라미터 사용하기
- qmark, named(추천) 방법

In [150]:

conn = sqlite3.connect('data/ch10_example.db')
cursor=conn.cursor()
cursor.execute('SELECT * FROM MEMBER')
cursor.fetchmany(size=6)

[('홍길동', 20, 'H@H.COM'), ('홍길동', 20, 'H@H.COM'), ('홍길동', 20, 'H@H.COM')]

In [152]:
#파라미터 사용하기 : qmark 방법 이용
name1=input("검색할 이름1:")
name2=input("검색할 이름2:")
names=[name1, name2] #튜플이나 리스트형 변수
cursor.execute('SELECT * FROM MEMBER WHERE NAME IN (?,?)',(name1,name2))
print(cursor.fetchall())
cursor.execute('SELECT * FROM MEMBER WHERE NAME IN (?,?)',names)
print(cursor.fetchall())

검색할 이름1:ㅁ
검색할 이름2:ㅁ
[]
[]


In [153]:
# 파라미터 사용하기 : named 방법 이용
name1 = input('검색할 이름은 : ')
cursor.execute('SELECT * FROM MEMBER WHERE NAME = :name',{'name':name1} )
members_list=cursor.fetchall()

검색할 이름은 : 홍길동


In [89]:
#member 테이블에 입력(사용자로부터 이름, 나이, 메일을 입력받아 DB에 insert하는 프로그램 수행)
try:    
    name=input('이름 : ')
    age=int(input('나이 : '))
    email=input('이메일 : ')
except:
    print("유효x")
    age=10
    email=input('이메일 : ')
data={'name':name,'age':age,'email':email}
cursor.execute('INSERT INTO MEMBER VALUES (:name, :age, :email)',data)
conn.commit()

이름 : 무무
나이 : ㅂ
유효x
이메일 : H@.com


In [93]:
cursor.execute('SELECT * FROM MEMBER')
cursor.fetchall()

[('홍길동', 20, 'H@H.COM'),
 ('홍길동', 20, 'H@H.COM'),
 ('홍길동', 20, 'H@H.COM'),
 ('홍길동', 20, 'H@H.COM'),
 ('홍길동', 20, 'H@H.COM'),
 ('홍길동', 20, 'H@H.COM'),
 ('k', 33, 'd'),
 ('홍길동', 20, 'H@H.COM'),
 ('홍길동', 20, 'H@H.COM'),
 ('이길동', 21, 'H@H.com'),
 ('무무', 10, 'H@.com'),
 ('홍길동', 20, 'A@A.com')]

In [78]:
members

[('홍길동', 20, 'H@H.COM'),
 ('홍길동', 20, 'H@H.COM'),
 ('홍길동', 20, 'H@H.COM'),
 ('홍길동', 20, 'H@H.COM'),
 ('홍길동', 20, 'H@H.COM'),
 ('홍길동', 20, 'H@H.COM')]

In [91]:
member_list

[{'name': '홍길동', 'age': 20, 'email': 'H@H.COM'},
 {'name': '홍길동', 'age': 20, 'email': 'H@H.COM'},
 {'name': '홍길동', 'age': 20, 'email': 'H@H.COM'},
 {'name': '홍길동', 'age': 20, 'email': 'H@H.COM'},
 {'name': '홍길동', 'age': 20, 'email': 'H@H.COM'},
 {'name': '홍길동', 'age': 20, 'email': 'H@H.COM'}]

In [162]:
class User:
    "이름, 나이, 메일이 있는 class"
    def __init__(self, name, age, email):
        self.name=name
        self.age=age
        self.email=email
    def __str__(self):
        return "{:4} {:3} {}".format(self.name, self.age, self.email)
    def as_dict(self):
        return {'name':self.name, 'age':self.age, 'email':email}
name = input('이름 : ')
age = int(input('나이 : '))
email = input('이메일 : ')
user=User(name,age,email)
cursor.execute("INSERT INTO MEMBER VALUES (:name, :age, :email)", user.as_dict())
conn.commit()

이름 : 3
나이 : 3
이메일 : 3


In [97]:
cursor.close()
conn.close()

In [82]:
import csv
try:
    with open("data/ch10_ex.csv","w", encoding="utf-8", newline="") as f:
        dict_writer = csv.DictWriter(f, fieldnames=['name','age','email'], quoting=csv.QUOTE_NONNUMERIC)
        dict_writer.writeheader()
        dict_writer.writerows(member_list)
except FileNotFoundError as e:
    print(e)

In [80]:
with open('data/ch10_ex.csv','w', encoding="utf-8", newline="") as f:
    writer = csv.writer(f, quoting=csv.QUOTE_NONNUMERIC)
    writer.writerows(members)
    

## 2절. 오라클 데이터베이스 연결

In [98]:
#colap같은것은  ! pip install cx_Oracle # ! <-- 이거 쓰면 cmd명령어사용가능

In [99]:
import cx_Oracle
oracle_dsn= cx_Oracle.makedsn(host="localhost", port=1521, sid="xe")
oracle_dsn

'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))'

In [111]:
# conn 얻어오는 방법 1
conn=cx_Oracle.connect(user="scott",password="tiger", dsn=oracle_dsn)
conn
#여기서 에러날 시에 VX_redist.x64.exe 인스톨

<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [108]:
# conn 얻어오는 방법 2
conn=cx_Oracle.connect("scott","tiger","localhost:1521/xe")
conn

<cx_Oracle.Connection to scott@localhost:1521/xe>

In [112]:
# cursor 얻어오기
cursor=conn.cursor()

In [114]:
emps=cursor.execute("SELECT * FROM EMP")

In [115]:
for emp in emps:
    print(emp)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [116]:
sql = "SELECT * FROM EMP WHERE ENAME=:ename"
ename=input("검색할 이름 : ")
cursor.execute(sql,{'ename':ename})
emp=cursor.fetchall()
if len(emp)==0:
    print('입력하신 이름의 데이터는 없습ㄴ디ㅏ.')
else:
    for e in emp:
        print(e)

검색할 이름 : SMITH
(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)


In [119]:
cursor.description

[('EMPNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 0),
 ('ENAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 10, 10, None, None, 1),
 ('JOB', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 9, 9, None, None, 1),
 ('MGR', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 1),
 ('HIREDATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 1),
 ('SAL', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('COMM', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('DEPTNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 3, None, 2, 0, 1)]

In [121]:
[row[0] for row in cursor.description]

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [125]:
data_df.columns

RangeIndex(start=0, stop=8, step=1)

In [126]:
#데이터베이스 검색 결과를 DataFrame으로
import pandas as pd
cursor.execute('SELECT * FROM EMP')
data=cursor.fetchall()
data_df=pd.DataFrame(data)
data_df.columns=[row[0] for row in cursor.description]
data_df

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [110]:
cursor.close()
conn.close()

conn.close()해도 
cursor 작동은 하는 듯.. 근데 conn사용 못함. 당연 commit도 안됌.